In [54]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from random import sample, choices, random
from typing import List, Tuple

In [55]:
def route_distance(route: List[Tuple[float, float, float]]) -> float:
    return sum(euclidean(route[i], route[i+1]) for i in range(len(route) - 1))


In [56]:
def initialize_population(pop_size, grouped_points) -> List[List[Tuple[float, float, float]]]:
    population = []
    for _ in range(pop_size):
        individual = []
        for group, points in grouped_points:
            points_list = points[['X', 'Y', 'Z']].values.tolist()
            individual.extend(sample(points_list, len(points_list))) 
        population.append(individual)
    return population


In [57]:
def fitness(route: List[Tuple[float, float, float]]) -> float:
    return -route_distance(route) 


In [58]:
def tournament_selection(population: List[List[Tuple[float, float, float]]], k=3) -> List[Tuple[float, float, float]]:
    candidates = sample(population, k) 
    candidates_fitness = [fitness(candidate) for candidate in candidates]
    return candidates[np.argmax(candidates_fitness)]  


In [59]:
def crossover(parent1: List[Tuple[float, float, float]], parent2: List[Tuple[float, float, float]]) -> List[Tuple[float, float, float]]:
    cut = np.random.randint(1, len(parent1) - 1)
    child = parent1[:cut] + parent2[cut:]
    return child


In [60]:
def mutate(individual: List[Tuple[float, float, float]], mutation_rate: float) -> List[Tuple[float, float, float]]:
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if random() < mutation_rate:
            j = np.random.randint(0, len(mutated_individual)) 
            mutated_individual[i], mutated_individual[j] = mutated_individual[j], mutated_individual[i]  # Trocar
    return mutated_individual


In [61]:
def generate_new_population(old_population: List[List[Tuple[float, float, float]]], mutation_rate, num_individuals, elite_size) -> List[List[Tuple[float, float, float]]]:
    new_population = sorted(old_population, key=fitness)[:elite_size]  
    while len(new_population) < num_individuals:
        parent1 = tournament_selection(old_population)
        parent2 = tournament_selection(old_population)
        child = crossover(parent1, parent2)
        child = mutate(child, mutation_rate)
        new_population.append(child)
    return new_population


In [62]:
def genetic_algorithm(points, pop_size, num_generations, mutation_rate, elite_size) -> List[Tuple[float, float, float]]:
    population = initialize_population(pop_size=pop_size, grouped_points=points )
    for _ in range(num_generations):
        population = sorted(population, key=fitness)[:elite_size]
        
        while len(population) < pop_size:
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            population.append(child)  
        
    best_individual = max(population, key=fitness)
    return best_individual

In [63]:
points_df = pd.read_csv('CaixeroGrupos.csv')

points_df.head()

grouped_points = points_df.groupby('GRUPO')

best_individual = genetic_algorithm(grouped_points, pop_size=100, num_generations=100, mutation_rate=0.001, elite_size=5)

print('Distância total da melhor rota encontrada:', route_distance(best_individual))
print('Melhor rota encontrada:', best_individual)
print('Ordem dos grupos na melhor rota encontrada:', [group for group, _ in grouped_points])

Distância total da melhor rota encontrada: 3445.564442648171
Melhor rota encontrada: [[0.0, 0.0, 0.0], [61.01040017660094, 61.429606927588154, 85.8483968667191], [59.34648610982673, 62.92556457589777, 89.19117086559613], [66.40142941712539, 62.483885084728875, 86.30086021060265], [62.884862630312256, 72.2182916006428, 83.35241506557027], [55.34685711692933, 66.46709009391914, 81.36121005153092], [64.00098183290838, 70.63403374401658, 85.64580053513586], [63.03365105954639, 79.94251611807414, 70.48944425948885], [73.83294557009027, 63.16968436773415, 72.04601495950786], [71.94010793516229, 67.9135383531448, 85.2265256306176], [61.12704724468078, 70.10563042578016, 72.96436670207373], [58.62724250078864, 72.82250115194253, 81.67774026878027], [68.98714931260807, 76.78530641218694, 84.95925400445373], [75.702661702107, 62.023977859627855, 80.6505978613277], [63.05083240802195, 60.626040502595934, 75.34855638517689], [70.86503128164641, 77.89125627494585, 76.4118920186515], [75.87545216343